# Random Forest

### Load Modules

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

### Load Dataset - include RepRisk data

In [2]:
# Load monthly firm characteristics raw data
df = pd.read_parquet('C:/Users/rafae/Documents/HSG/Master Thesis/Data/Final/data07_model_input.parquet')
df = df.sort_values(by=['YM', 'permno'])
df = df.set_index(['year', 'YM', 'permno'])
df

gvkey  reprisk_id         prc         vol         mve_m  \
year YM      permno                                                             
2007 2007-01 10025    11903       37172   45.320000      8086.0  3.700557e+05   
             10026    12825       12684   39.689999      7613.0  7.653725e+05   
             10042    12139        4832    0.720000     26008.0  3.598898e+04   
             10078    12136        1719    6.130000  11333293.0  2.390900e+07   
             10104    12142        4413   16.430000   7234361.0  8.892640e+07   
...                     ...         ...         ...         ...           ...   
2021 2021-12 93304   184167       91339   36.750000    183303.0  1.695898e+06   
             93373   184323       74074    3.020000    528712.0  2.065325e+05   
             93374   184899       64442   74.510002    159495.0  8.587073e+06   
             93423    10567       22547   39.490002    254917.0  3.661156e+06   
             93436   184996       24952  936.719971   6361641.0  1.092218e+09   

                       absacc       acc    aeavol       age       agr  ...  \
year YM      permno                                                    ...   
2007 2007-01 10025   0.698728 -0.745547 -0.646819  0.457506 -0.979644  ...   
             10026   0.577608 -0.635623 -0.393384  0.457506  0.118575  ...   
             10042   0.990840 -0.989822 -0.894148  0.457506 -0.989822  ...   
             10078   0.654962 -0.711959 -0.128753  0.905344 -0.147074  ...   
             10104  -0.014758 -0.107379  0.780153  0.905344  0.770992  ...   
...                       ...       ...       ...       ...       ...  ...   
2021 2021-12 93304  -0.855522  0.706269 -0.871045 -0.500896  0.613134  ...   
             93373  -0.213134  0.143881  0.663284 -0.500896 -0.875821  ...   
             93374  -0.514030  0.423284 -0.223881 -0.500896  0.328955  ...   
             93423   0.242985 -0.303881  0.560597  0.143881 -0.584478  ...   
             93436   0.584478 -0.620299 -0.362388 -0.500896  0.856716  ...   

                     sic2_73  sic2_75  sic2_78  sic2_79  sic2_80  sic2_81  \
year YM      permno                                                         
2007 2007-01 10025         0        0        0        0        0        0   
             10026         0        0        0        0        0        0   
             10042         0        0        0        0        0        0   
             10078         0        0        0        0        0        0   
             10104         1        0        0        0        0        0   
...                      ...      ...      ...      ...      ...      ...   
2021 2021-12 93304         0        0        0        0        0        0   
             93373         0        0        0        0        0        0   
             93374         0        0        0        0        0        0   
             93423         0        0        0        1        0        0   
             93436         0        0        0        0        0        0   

                     sic2_82  sic2_83  sic2_87  sic2_99  
year YM      permno                                      
2007 2007-01 10025         0        0        0        0  
             10026         0        0        0        0  
             10042         0        0        0        0  
             10078         0        0        0        0  
             10104         0        0        0        0  
...                      ...      ...      ...      ...  
2021 2021-12 93304         0        0        0        0  
             93373         0        0        0        0  
             93374         0        0        0        0  
             93423         0        0        0        0  
             93436         0        0        0        0  

[338288 rows x 188 columns]

In [3]:
# Select only relevant columns for X and Y (exclude industry & reprisk rating dummy for now)
info_vars = ['YM', 'year', 'permno', 'gvkey', 'reprisk_id', 'prc', 'vol', 'mve_m']
X_vars = ['absacc', 'acc', 'aeavol', 'age', 'agr', 'baspread', 'beta', 'betasq', 'bm', 'bm_ia', 'cash', 'cashdebt',
          'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chmom', 'chpmia', 'chtx', 'cinvest',
          'convind', 'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 'ear', 'egr', 'ep', 'gma', 'grcapx', 'grltnoa',
          'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'lev', 'lgr', 'maxret', 'mom12m', 'mom1m', 'mom36m',
          'mom6m', 'ms', 'mve', 'mve_ia', 'nincr', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr',
          'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc',
          'pricedelay', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'retvol', 'roaq', 'roavol', 'roeq',
          'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol',
          'std_turn', 'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade']
sic2_vars = [col for col in df if col.startswith('sic2')]
reprisk_vars = ['country_sector_average', 'country_sector_average_01', 'current_rri', 'current_rri_01',
                'peak_rri', 'peak_rri_01', 'trend_rri', 'trend_rri_01']
reprisk_rating_vars = [col for col in df if col.startswith('reprisk_rating')]
Y_vars = ['ret', 'ret_wins', 'ret_ex']

# X
X = df[X_vars + sic2_vars + reprisk_vars + reprisk_rating_vars]

# Y
Y = df[['ret_ex']]

X.shape, Y.shape

((338288, 180), (338288, 1))

### Hyperparameter Optimization: 4-Fold CV (12y/4 = 3y) and 3y Test

In [4]:
# Training (12y - 80%) and Test set (3y - 20%)
X_trai = X.loc['2007':'2018']
Y_trai = Y.loc['2007':'2018']

X_test = X.loc['2019':'2021']
Y_test = Y.loc['2019':'2021']

In [5]:
# 4-Fold cross validation (9y training and 3y validation)
K_FOLDs = 4
YEARS = list(X_trai.index.unique(level='year').astype('str')) 
TOT = len(YEARS)
TRA = int(TOT* (K_FOLDs-1) / K_FOLDs)
OFF = TOT - TRA

for FOLD in range(K_FOLDs):
    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
    TRAI = [x for x in YEARS if x not in VALI]
    print(VALI, TRAI)    

['2007', '2008', '2009'] ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
['2010', '2011', '2012'] ['2007', '2008', '2009', '2013', '2014', '2015', '2016', '2017', '2018']
['2013', '2014', '2015'] ['2007', '2008', '2009', '2010', '2011', '2012', '2016', '2017', '2018']
['2016', '2017', '2018'] ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']


In [6]:
# Model
model_name = 'RF_esg'

In [7]:
# Hyperparameter optimization
Y_val_preds = Y_trai.copy()
Y_test_preds = Y_test.copy()
results = []

model_ix = 0
for depth in [1, 2, 3, 4, 5]:
    for n_features in [3, 5, 10, 20, 30]:
        print("•", model_ix, end=' ')
                    
        for FOLD in range(K_FOLDs):
            VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
            TRAI = [x for x in YEARS if x not in VALI]
            
            # Reset seeds
            np.random.seed(0)
            random.seed(0)
            
            # Create model
            RF = RandomForestRegressor(max_depth=depth, max_features=n_features, n_estimators=300, n_jobs=-1, random_state=0)

            # Fit the model
            RF.fit(X_trai.loc[TRAI], Y_trai.loc[TRAI].values.ravel()) 

            # Calculate validation predictions
            val_preds = RF.predict(X_trai.loc[VALI])
            Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = val_preds
            val_loss = mean_squared_error(Y_trai.loc[VALI], val_preds)
                        
            # Calculate predictions for test data, if FOLD = 0
            if FOLD==0:
                Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = RF.predict(X_test)
            
            # Append results
            results.append({
                'model_ix'  :model_ix,
                'depth'     :depth,
                'n_features':n_features,
                'fold'      :FOLD,
                'val_loss'  :val_loss
            })
            
        model_ix += 1

• 0 • 1 • 2 • 3 • 4 • 5 • 6 • 7 • 8 • 9 • 10 • 11 • 12 • 13 • 14 • 15 • 16 • 17 • 18 • 19 • 20 • 21 • 22 • 23 • 24 

In [8]:
# Save Y_val_preds
Y_val_preds.to_csv(r'%s/%s_val_preds.csv'%(model_name, model_name))
Y_val_preds

ret_ex  RF_esg_0  RF_esg_1  RF_esg_2  RF_esg_3  \
year YM      permno                                                     
2007 2007-01 10025  -0.031894  0.009790  0.010015  0.010498  0.010791   
             10026  -0.042317  0.010239  0.010270  0.010355  0.010435   
             10042  -0.125751  0.004497  0.001765  0.001720 -0.000379   
             10078  -0.080607  0.009668  0.009772  0.010256  0.011440   
             10104  -0.046341  0.010510  0.010704  0.011138  0.011880   
...                       ...       ...       ...       ...       ...   
2018 2018-12 93420   0.086508  0.004213  0.003672  0.002667  0.000798   
             93422   0.466817  0.003161  0.002777  0.002824  0.002894   
             93423   0.105036  0.005468  0.005292  0.004760  0.003475   
             93429  -0.048712  0.004252  0.004535  0.004630  0.004966   
             93436  -0.079564  0.004093  0.003552  0.002523  0.001110   

                     RF_esg_4  RF_esg_5  RF_esg_6  RF_esg_7  RF_esg_8  ...  \
year YM      permno                                                    ...   
2007 2007-01 10025   0.010915  0.010361  0.010415  0.011136  0.011554  ...   
             10026   0.010643  0.010538  0.010754  0.010677  0.010726  ...   
             10042   0.000805  0.000145 -0.003081 -0.002963 -0.003482  ...   
             10078   0.011904  0.009669  0.009986  0.010516  0.012562  ...   
             10104   0.012205  0.011122  0.011383  0.011678  0.012870  ...   
...                       ...       ...       ...       ...       ...  ...   
2018 2018-12 93420  -0.000542  0.002908  0.001682  0.000204 -0.002481  ...   
             93422   0.002628  0.000387 -0.001425 -0.001709 -0.005517  ...   
             93423   0.002187  0.005299  0.005478  0.005105  0.003448  ...   
             93429   0.004873  0.004011  0.004018  0.002244  0.002282  ...   
             93436  -0.000305  0.002610  0.001674  0.000114 -0.001802  ...   

                     RF_esg_15  RF_esg_16  RF_esg_17  RF_esg_18  RF_esg_19  \
year YM      permno                                                          
2007 2007-01 10025    0.011170   0.011474   0.011962   0.011869   0.010781   
             10026    0.011135   0.011405   0.011474   0.010796   0.009681   
             10042   -0.010075  -0.009621  -0.017643  -0.007705  -0.005516   
             10078    0.010371   0.010626   0.011766   0.012318   0.012778   
             10104    0.011633   0.012045   0.012528   0.012800   0.013246   
...                        ...        ...        ...        ...        ...   
2018 2018-12 93420   -0.001715  -0.002927  -0.005364  -0.006727  -0.005087   
             93422   -0.004067  -0.008159  -0.011214  -0.017299  -0.013828   
             93423    0.005863   0.005361   0.004966   0.004682   0.005356   
             93429    0.003165   0.001536  -0.000874  -0.000665   0.003727   
             93436    0.000413  -0.001054  -0.002690  -0.003474  -0.002739   

                     RF_esg_20  RF_esg_21  RF_esg_22  RF_esg_23  RF_esg_24  
year YM      permno                                                         
2007 2007-01 10025    0.010885   0.011161   0.012786   0.011061   0.011302  
             10026    0.011096   0.010914   0.011786   0.010206   0.009892  
             10042   -0.011127  -0.020470  -0.023102  -0.011273  -0.007726  
             10078    0.009875   0.010802   0.011971   0.013224   0.013123  
             10104    0.011911   0.012146   0.012477   0.013100   0.013387  
...                        ...        ...        ...        ...        ...  
2018 2018-12 93420   -0.001987  -0.006609  -0.006713  -0.007179  -0.006893  
             93422   -0.006909  -0.010737  -0.016711  -0.022461  -0.020140  
             93423    0.005717   0.005087   0.005643   0.005960   0.005616  
             93429    0.002739  -0.002173  -0.002932  -0.004134  -0.002973  
             93436   -0.000662  -0.001327  -0.003622  -0.002052  -0.002717  

[276564 rows x 26 columns]

In [9]:
# Save Y_test_preds
Y_test_preds.to_csv(r'%s/%s_test_preds.csv'%(model_name, model_name))
Y_test_preds

ret_ex  RF_esg_0  RF_esg_1  RF_esg_2  RF_esg_3  \
year YM      permno                                                     
2019 2019-01 10026   0.004225  0.010343  0.010469  0.010647  0.010782   
             10104   0.036026  0.010306  0.010271  0.010258  0.010236   
             10107   0.075381  0.010226  0.010054  0.009807  0.009216   
             10138   0.072777  0.010462  0.010705  0.011116  0.011880   
             10145   0.076596  0.010277  0.010308  0.010304  0.010260   
...                       ...       ...       ...       ...       ...   
2021 2021-12 93304  -0.096386  0.010020  0.010018  0.009877  0.009827   
             93373  -0.019481  0.008485  0.007403  0.004105  0.001974   
             93374  -0.047552  0.010322  0.010542  0.010736  0.010860   
             93423  -0.072569  0.011312  0.011417  0.010653  0.011077   
             93436  -0.113609  0.009701  0.009721  0.009614  0.009500   

                     RF_esg_4  RF_esg_5  RF_esg_6  RF_esg_7  RF_esg_8  ...  \
year YM      permno                                                    ...   
2019 2019-01 10026   0.010941  0.010765  0.010938  0.011047  0.011306  ...   
             10104   0.009855  0.010613  0.010506  0.011325  0.010806  ...   
             10107   0.008631  0.010413  0.010044  0.009993  0.009289  ...   
             10138   0.012205  0.010976  0.011325  0.012360  0.012779  ...   
             10145   0.009878  0.010648  0.010624  0.011473  0.010827  ...   
...                       ...       ...       ...       ...       ...  ...   
2021 2021-12 93304   0.009744  0.010729  0.010175  0.009564  0.009224  ...   
             93373   0.000007  0.009672  0.008199  0.001313 -0.002696  ...   
             93374   0.010981  0.010814  0.011081  0.011285  0.011392  ...   
             93423   0.011128  0.011706  0.012655  0.013363  0.014354  ...   
             93436   0.008885  0.009741  0.009861  0.010363  0.009656  ...   

                     RF_esg_15  RF_esg_16  RF_esg_17  RF_esg_18  RF_esg_19  \
year YM      permno                                                          
2019 2019-01 10026    0.011318   0.011336   0.011161   0.011462   0.010956   
             10104    0.011148   0.010843   0.012323   0.012718   0.013720   
             10107    0.010573   0.010123   0.009812   0.009363   0.009115   
             10138    0.011394   0.012012   0.013493   0.015287   0.015904   
             10145    0.011384   0.011077   0.012598   0.013236   0.013545   
...                        ...        ...        ...        ...        ...   
2021 2021-12 93304    0.010335   0.009089   0.008991   0.008445   0.008223   
             93373    0.008409   0.007147  -0.001763  -0.007626  -0.010402   
             93374    0.011560   0.011702   0.012060   0.012014   0.011390   
             93423    0.013591   0.015928   0.017378   0.014714   0.013361   
             93436    0.010731   0.010194   0.010729   0.008459   0.007341   

                     RF_esg_20  RF_esg_21  RF_esg_22  RF_esg_23  RF_esg_24  
year YM      permno                                                         
2019 2019-01 10026    0.011434   0.011753   0.011493   0.011494   0.011257  
             10104    0.011313   0.011554   0.012507   0.013490   0.016095  
             10107    0.010333   0.010605   0.010032   0.009229   0.009537  
             10138    0.011822   0.012105   0.014424   0.015670   0.018597  
             10145    0.011353   0.011769   0.012923   0.013966   0.016112  
...                        ...        ...        ...        ...        ...  
2021 2021-12 93304    0.010817   0.010861   0.008531   0.010949   0.006860  
             93373    0.008596   0.003138  -0.005611  -0.008706  -0.013477  
             93374    0.011398   0.011922   0.012003   0.011881   0.011537  
             93423    0.015848   0.017146   0.016999   0.014591   0.013386  
             93436    0.009955   0.010766   0.011084   0.009895   0.005966  

[61724 rows x 26 columns]

In [10]:
# Result overview
table = pd.DataFrame(results)
table = table.groupby(['model_ix', 'depth', 'n_features']).mean().sort_values('val_loss')
table.to_csv(r'%s/%s_results.csv'%(model_name, model_name))

np.sqrt(table[['val_loss']].head(20)) * 100

,,,val_loss
model_ix,depth,n_features,
20,5,3,12.929967
21,5,5,12.932073
16,4,5,12.932586
15,4,3,12.932664
11,3,5,12.933248
6,2,5,12.933434
5,2,3,12.933979
10,3,3,12.934303
7,2,10,12.934701


## Model Selection: Maximize SR on LS-Portfolio on validation data

In [11]:
# Model
model_name = 'RF_esg'

# Get predictions
val = pd.read_csv(r'%s/%s_val_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])
test = pd.read_csv(r'%s/%s_test_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])

# Get overview table
table = pd.read_csv(r'%s/%s_results.csv'%(model_name, model_name)).sort_values(['model_ix'])

# Extract hyperparameters
list_depth = list(pd.unique(table['depth']))
list_n_features = list(pd.unique(table['n_features']))

In [12]:
# Loop over hyperparameter combinations and calculate SR
val_results = []
hyper_comb = 0
for depth in list_depth:
    for n_features in list_n_features:
        # Get relevant model_ix
        model_ix = table[(table['depth'] == depth) & (table['n_features'] == n_features)]
        model_ix = model_ix['model_ix']
        # Select relevant returns
        val_ret = val[['%s_%d'%(model_name, model_ix)]]
        comb = val[['ret_ex']].copy()
        comb['ret_pred'] = val_ret
        comb = comb.reset_index()
        # Sort the data by predicted returns and divide the data into quintiles
        comb['quintile'] = comb.groupby(['YM'])['ret_pred'].transform(lambda x: pd.qcut(x.rank(method='first'), 5, labels=np.arange(1,6)))
        # Calculate the mean return for each YM
        comb_mean = comb.groupby(['YM', 'quintile']).agg(ret_ex = ('ret_ex', 'mean'))
        # Add LS-Strategy
        comb_mean = comb_mean[['ret_ex']].unstack().add_prefix('Q')
        comb_mean.columns = comb_mean.columns.droplevel(0)
        comb_mean.columns.name = None
        comb_mean['LS'] = comb_mean['Q5'] - comb_mean['Q1']
        # Calculate the average return, standard deviation and Sharpe Ratio (annualized) per Quintile
        summary = pd.DataFrame()
        summary['mean'] = comb_mean.mean()
        summary['std'] = comb_mean.std()
        summary['SR'] = summary['mean'] / summary['std'] * np.sqrt(12)
        # Append results
        val_results.append({
            'hyper_comb':hyper_comb,
            'depth'     :depth,
            'n_features':n_features,
            'SR_Q1'     :summary.loc['Q1','SR'],
            'SR_Q5'     :summary.loc['Q5','SR'],
            'SR_LS'     :summary.loc['LS','SR'],
            'Mean_Q1'   :summary.loc['Q1','mean'],
            'Mean_Q5'   :summary.loc['Q5','mean'],
            'Mean_LS'   :summary.loc['LS','mean'],
        })
        hyper_comb += 1  

# Save results
val_results = pd.DataFrame(val_results)
val_results.to_csv(r'results/{}_val.csv'.format(model_name))        

In [13]:
# Select best hyperparemeters (max. SR_LS)
list_depth = list(pd.unique(table['depth']))
list_n_features = list(pd.unique(table['n_features']))

val_results = val = pd.read_csv(r'results/{}_val.csv'.format(model_name))
val_results = val_results.set_index(['depth', 'n_features']).sort_values(['SR_LS'], ascending=False)
depth_opt, n_features_opt = tuple([x for x in val_results.iloc[0].name[0:]])
print('Optimal depth:     ', depth_opt)
print('Optimal n_features:', n_features_opt)
val_results

Optimal depth:      5
Optimal n_features: 3


Unnamed: 0  hyper_comb     SR_Q1     SR_Q5     SR_LS  \
depth n_features                                                         
5     3                   20          20  0.015407  0.582147  0.803321   
2     3                    5           5  0.021562  0.607434  0.780620   
4     3                   15          15  0.028536  0.575314  0.746860   
1     3                    0           0  0.039701  0.600089  0.737493   
3     3                   10          10  0.051275  0.599009  0.731987   
      5                   11          11  0.051374  0.602795  0.714677   
1     5                    1           1  0.064345  0.620271  0.708147   
4     5                   16          16  0.068235  0.602158  0.684804   
2     5                    6           6  0.068537  0.608283  0.663177   
5     5                   21          21  0.081111  0.589810  0.656992   
      20                  23          23  0.108247  0.497608  0.613960   
4     10                  17          17  0.119670  0.525146  0.552107   
1     10                   2           2  0.138658  0.584929  0.530100   
4     30                  19          19  0.170772  0.492893  0.524577   
2     10                   7           7  0.122683  0.550019  0.514910   
5     30                  24          24  0.146067  0.494501  0.504591   
      10                  22          22  0.122111  0.504943  0.498957   
3     20                  13          13  0.143989  0.480650  0.495805   
      10                  12          12  0.150546  0.544833  0.491441   
1     20                   3           3  0.212970  0.576138  0.472414   
4     20                  18          18  0.162581  0.479468  0.468015   
2     20                   8           8  0.190252  0.542287  0.455279   
1     30                   4           4  0.231050  0.580618  0.444444   
2     30                   9           9  0.222553  0.546408  0.407137   
3     30                  14          14  0.199399  0.467151  0.388418   

                   Mean_Q1   Mean_Q5   Mean_LS  
depth n_features                                
5     3           0.000342  0.009027  0.008685  
2     3           0.000483  0.009230  0.008748  
4     3           0.000632  0.008838  0.008206  
1     3           0.000883  0.009060  0.008177  
3     3           0.001124  0.009104  0.007980  
      5           0.001124  0.009048  0.007924  
1     5           0.001411  0.009251  0.007840  
4     5           0.001510  0.009194  0.007685  
2     5           0.001525  0.009050  0.007526  
5     5           0.001783  0.009065  0.007281  
      20          0.002315  0.008300  0.005984  
4     10          0.002593  0.008330  0.005737  
1     10          0.002975  0.008691  0.005715  
4     30          0.003608  0.008254  0.004646  
2     10          0.002674  0.008314  0.005639  
5     30          0.003111  0.008060  0.004950  
      10          0.002672  0.008135  0.005464  
3     20          0.003037  0.007783  0.004746  
      10          0.003241  0.008416  0.005176  
1     20          0.004381  0.008872  0.004490  
4     20          0.003452  0.007914  0.004462  
2     20          0.003979  0.008470  0.004491  
1     30          0.004787  0.008962  0.004175  
2     30          0.004708  0.008613  0.003905  
3     30          0.004174  0.007695  0.003521

## Out-of-Sample: Get Predictions of model with best hyperparameter combination

In [17]:
# Get relevant model_ix
model_ix = table[(table['depth'] == depth_opt) & (table['n_features'] == n_features_opt)]
model_ix = model_ix['model_ix']

# Select relevant returns
test_ret = test.copy()
test_ret['y_pred'] = test[['%s_%d'%(model_name, model_ix)]]

# Save predictions
test_ret = test_ret.reset_index()
test_ret = test_ret[['YM', 'permno', 'y_pred']]
test_ret.to_csv(r'results/{}_predictions.csv'.format(model_name), index=False)